In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add

def residual_block(input_tensor, filters):
    r = Conv2D(filters, 3, padding="same", use_bias=False)(input_tensor)
    r = BatchNormalization()(r)
    r = Activation("relu")(r)

    r = Conv2D(filters, 3, padding="same", use_bias=False)(r)
    r = BatchNormalization()(r)

    output = Add()([input_tensor, r])
    output = Activation("relu")(output)

    return output


In [ ]:
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

inputs = Input(shape=(224, 224, 3))

# ---- Block 1 ----
x1 = Conv2D(32, 3, padding="same", use_bias=False)(inputs)
x1 = BatchNormalization()(x1)
x1 = Activation("relu")(x1)

# ---- Block 2 ----
x2 = Conv2D(32, 3, padding="same", use_bias=False)(x1)
x2 = BatchNormalization()(x2)
x2 = Activation("relu")(x2)
x2 = residual_block(x2, 32)

# ---- Block 3 (Downsampling) ----
x3 = Conv2D(64, 3, strides=2, padding="same", use_bias=False)(x2)
x3 = BatchNormalization()(x3)
x3 = Activation("relu")(x3)

# ---- Block 4 ----
x4 = Conv2D(64, 3, padding="same", use_bias=False)(x3)
x4 = BatchNormalization()(x4)
x4 = Activation("relu")(x4)
x4 = residual_block(x4, 64)

# ---- Classification Head ----
gap = GlobalAveragePooling2D()(x4)
drop = Dropout(0.4)(gap)
outputs = Dense(10, activation="softmax")(drop)

model = Model(inputs, outputs)
model.summary()
